# The Collatz Conjecture

In [ ]:
import sys
print(sys.version)

3.7.2 (default, Dec 29 2018, 00:00:04) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


## Naive Check Up To 1,000,000

First, let's create a simple dataclass to hold the results of a collatz run.

In [ ]:
from dataclasses import dataclass

@dataclass
class Collatz:
    __slots__ = ["n", "pathlen", "below", "below_val", "maxn", "maxn_val"]
    n: int
    pathlen: int
    below: int
    below_val: int
    maxn: int
    maxn_val: int
        
    def __str__(self):
        return f"A={self.n} path={self.pathlen} max={self.maxn_val}@{self.maxn} drop={self.below_val}@{self.below}"


Does it work?

In [ ]:
c = Collatz(77671, 231, 171, 59173, 71, 1570824736)
print(c)

A=77671 path=231 max=1570824736@71 drop=59173@171


Now let's collatz a number and log all the details:
- path len
- what was the max hailstone number and when did it happen (maxn)
- when did the deries drop below the initial number (below)

In [ ]:
VERBOSE = 0

# VERBOSE default will be evaluated at compile time
def collatz(n, verbose=VERBOSE):
    cnt = 0
    n0 = n
    below = -1
    below_val = -1
    maxn = -1
    maxn_val = n
    if verbose >= 2: print(n, end=" ")
        
    while n != 1:
        if n % 2 == 1:
            n = 3 * n + 1
        else:
            n = int(n / 2)
        cnt += 1
        # record when the series goes into mathematical induction
        if below == -1 and n < n0:
            below = cnt
            below_val = n
        # record the peak hailstone number
        if n > maxn_val:
            maxn = cnt
            maxn_val = n
            if verbose >= 2: print("^", end="")
        if verbose >= 2: print(n, end=" ")

    if verbose >= 2: print()
    if verbose >= 1: print(f"path={cnt} max={maxn_val}@{maxn} drop={below_val}@{below}")
    return Collatz(n0, cnt, below, below_val, maxn, maxn_val)


Abuse spot check as unit test:

In [ ]:
assert collatz(77671, verbose=0) == Collatz(77671, 231, 171, 59173, 71, 1570824736)

Calculate some series and store them in an array. Also let's get a first impression on performance.

In [ ]:
from time import time

MAX = 1000000
print("calculating up to", MAX)
t0 = time()
a = list()
a.append(0)
for i in range(1,MAX):
    a.append(collatz(i, verbose=0))
t1 = time()
print("%.1fs" % (t1 - t0,))


calculating up to 1000000
32.3s


We already know that residual classes will play a role, so let's add them to the output:

In [ ]:
def resclasses(n):
    a = list()
    for p in [3,5,7,11,13,17,19]:
        a.append(str(n % p))
    s = ', '.join(a)
    return f"res=({s})"


In [ ]:
resclasses(27)

'res=(0, 2, 6, 5, 1, 10, 8)'

Let's get a first idea what we have in our hands:

In [ ]:
maxn_val = 0
pathlen = 0
for c in a:
    if not isinstance(c, Collatz): # skip the 0
        continue
    if c.n == 1:
        continue
    do_print = False
    if c.pathlen > pathlen:
        pathlen = c.pathlen
        do_print = True
    if do_print:
        print(c, resclasses(c.n))
        

A=2 path=1 max=2@-1 drop=1@1 res=(2, 2, 2, 2, 2, 2, 2)
A=3 path=7 max=16@3 drop=2@6 res=(0, 3, 3, 3, 3, 3, 3)
A=6 path=8 max=16@4 drop=3@1 res=(0, 1, 6, 6, 6, 6, 6)
A=7 path=16 max=52@5 drop=5@11 res=(1, 2, 0, 7, 7, 7, 7)
A=9 path=19 max=52@8 drop=7@3 res=(0, 4, 2, 9, 9, 9, 9)
A=18 path=20 max=52@9 drop=9@1 res=(0, 3, 4, 7, 5, 1, 18)
A=25 path=23 max=88@6 drop=19@3 res=(1, 0, 4, 3, 12, 8, 6)
A=27 path=111 max=9232@77 drop=23@96 res=(0, 2, 6, 5, 1, 10, 8)
A=54 path=112 max=9232@78 drop=27@1 res=(0, 4, 5, 10, 2, 3, 16)
A=73 path=115 max=9232@81 drop=55@3 res=(1, 3, 3, 7, 8, 5, 16)
A=97 path=118 max=9232@84 drop=73@3 res=(1, 2, 6, 9, 6, 12, 2)
A=129 path=121 max=9232@87 drop=97@3 res=(0, 4, 3, 8, 12, 10, 15)
A=171 path=124 max=9232@90 drop=145@8 res=(0, 1, 3, 6, 2, 1, 0)
A=231 path=127 max=9232@93 drop=124@19 res=(0, 1, 0, 0, 10, 10, 3)
A=313 path=130 max=9232@96 drop=235@3 res=(1, 3, 5, 5, 1, 7, 9)
A=327 path=143 max=9232@109 drop=250@34 res=(0, 2, 5, 8, 2, 4, 4)
A=649 path=144 max=9232@

## 100 Mio. Numbers In 3 Minutes

Ok, we need more numbers. But for that we should reduce memory consumption and compute time: only compute until it drops below start value.

In [ ]:
VERBOSE = 0

@dataclass
class Collatz2:
    __slots__ = ["n", "drop"]
    n: int
    drop: int

    def __str__(self):
        return f"A={self.n} drops@{self.drop}"

# VERBOSE default will be evaluated at compile time
def collatz2(n, verbose=VERBOSE):
    cnt = 0
    n0 = n
    below = -1
    below_val = -1
    maxn = -1
    maxn_val = n
    if verbose >= 2: print(n, end=" ")
        
    while n >= n0:
        if n % 2 == 1:
            n = 3 * n + 1
        else:
            n = int(n / 2)
        cnt += 1
        # record when the series goes into mathematical induction
        if below == -1 and n < n0:
            below = cnt
            below_val = n
        # record the peak hailstone number
        if n > maxn_val:
            maxn = cnt
            maxn_val = n
            if verbose >= 2: print("^", end="")
        if verbose >= 2: print(n, end=" ")

    if verbose >= 2: print()
    if verbose >= 1: print(f"path={cnt} max={maxn_val}@{maxn} drop={below_val}@{below}")
    return Collatz2(n0, below)


In [ ]:
collatz2(27)

Collatz2(n=27, drop=96)

Now let's work thru a bigger chunk of numbers and avoid storing too many results:

In [ ]:
from time import time

MAX = 100_000_000
# 100 mio = 1E+8 -> 3m20s
DECILE = int(MAX / 10)
print("calculating up to", MAX)
t0 = time()
a2 = list()
drop = 0
for i in range(2,MAX+1):
#    print(i, end=" ")
    if i % DECILE == 0:
        percent = int(i/DECILE)*10
        t = "%.1fs" % (time() - t0,)
        print(f"--{percent}% {t}--", end=" ")
    c = collatz2(i, verbose=0)
    if c.drop > drop:
        drop = c.drop
        print(f"{c.n:,d}", end=" ")
        a2.append(c)
print()
t1 = time()
print("total elapsed: %.1fs" % (t1 - t0,))


calculating up to 100000000
2 3 7 27 703 10,087 35,655 270,271 362,343 381,727 626,331 1,027,431 1,126,015 8,088,063 --10% 19.2s-- 13,421,671 --20% 39.6s-- 20,638,335 26,716,671 --30% 59.3s-- --40% 80.0s-- --50% 100.6s-- 56,924,955 --60% 121.4s-- 63,728,127 --70% 142.0s-- --80% 162.3s-- --90% 182.4s-- --100% 202.9s-- 
total elapsed: 202.9s


This will, of course, produce much less storage:

In [ ]:
for c in a2:
    if not isinstance(c, Collatz2): # skip the 0s
        continue
    print(f"{c.n:,d}", c.drop, resclasses(c.n))
    

2 1 res=(2, 2, 2, 2, 2, 2, 2)
3 6 res=(0, 3, 3, 3, 3, 3, 3)
7 11 res=(1, 2, 0, 7, 7, 7, 7)
27 96 res=(0, 2, 6, 5, 1, 10, 8)
703 132 res=(1, 3, 3, 10, 1, 6, 0)
10,087 171 res=(1, 2, 0, 0, 12, 6, 17)
35,655 220 res=(0, 0, 4, 4, 9, 6, 11)
270,271 267 res=(1, 1, 1, 1, 1, 5, 15)
362,343 269 res=(0, 3, 2, 3, 7, 5, 13)
381,727 282 res=(1, 2, 3, 5, 8, 9, 17)
626,331 287 res=(0, 1, 6, 2, 4, 0, 15)
1,027,431 298 res=(0, 1, 6, 9, 2, 2, 6)
1,126,015 365 res=(1, 0, 2, 0, 7, 3, 18)
8,088,063 401 res=(0, 3, 4, 5, 9, 7, 10)
13,421,671 468 res=(1, 1, 4, 10, 3, 1, 14)
20,638,335 476 res=(0, 0, 4, 3, 3, 12, 3)
26,716,671 486 res=(0, 1, 2, 3, 7, 15, 11)
56,924,955 502 res=(0, 0, 3, 10, 9, 13, 5)
63,728,127 613 res=(0, 2, 1, 1, 8, 6, 18)


"Even longer" paths become **very** rare out there...